In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import linear_model, metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
import time
import os

In [2]:
X = pd.read_csv('train.csv')

In [3]:
X.head()

,Allied_Nations,Diplomatic_Meetings_Set,Percent_Of_Forces_Mobilized,Hostile_Nations,Active_Threats,Inactive_Threats,Citizen_Fear_Index,Closest_Threat_Distance(km),Aircraft_Carriers_Responding,Troops_Mobilized(thousands),DEFCON_Level,ID
0,16,1,0.67,3,6.0,25.0,0.787812,324.36,1,862400.0,3,7570
1,8,1,0.11,2,38.0,48.0,0.419236,359.34,0,959200.0,4,12128
2,9,1,0.49,3,32.0,69.0,0.582232,332.84,1,836000.0,3,2181
3,7,0,0.30,2,31.0,52.0,0.589574,358.28,1,924000.0,3,5946
4,8,1,0.12,5,13.0,42.0,0.552863,364.64,0,968000.0,2,9054


In [4]:
X.columns = X.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [5]:
X.columns

Index(['allied_nations', 'diplomatic_meetings_set',
       'percent_of_forces_mobilized', 'hostile_nations', 'active_threats',
       'inactive_threats', 'citizen_fear_index', 'closest_threat_distancekm',
       'aircraft_carriers_responding', 'troops_mobilizedthousands',
       'defcon_level', 'id'],
      dtype='object')

In [6]:
y = X['defcon_level']

In [7]:
X.drop(columns=['defcon_level'],inplace=True)

In [8]:
print(X.shape)

(10000, 11)


In [9]:
print(y.shape)

(10000,)


In [11]:
columns = ['diplomatic_meetings_set','hostile_nations','aircraft_carriers_responding']

In [12]:
X.drop(columns = ['diplomatic_meetings_set','hostile_nations','aircraft_carriers_responding'],axis=1,inplace=True)

In [10]:
print(X.shape)

(10000, 11)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20,random_state=101)

In [17]:
gb = GradientBoostingClassifier(n_estimators = 100, learning_rate= 0.03, max_depth= 10, max_features = 6, min_samples_split=1250)
# gb = GradientBoostingClassifier(n_estimators = 125, learning_rate= 0.05, max_depth= 5, max_features = 5, min_samples_split=1250)

In [18]:
start_gb = time.time()

gb_scores = cross_val_score(gb, X, y, cv=10, scoring='accuracy')
print('The accuracy of the Random forest classifier with 10 fold cross-validation is ' + str(gb_scores.mean()))

end_gb =time.time()
gb_time = (end_gb-start_gb)/60
print('The time taken for the classifier for 10 fold cross validation is ' + str(gb_time))


The accuracy of the Random forest classifier with 10 fold cross-validation is 0.5882043395043395
The time taken for the classifier for 10 fold cross validation is 0.6543980518976847


In [16]:
gb_cv_mean = np.mean(gb_scores)
gb_cv_variance = np.var(gb_scores)

print('gb Mean score : ', gb_cv_mean)
print('gb Score variance : ', gb_cv_variance)

gb Mean score :  0.574903333903334
gb Score variance :  0.0002006773848363602


In [48]:
gb.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=11,
                           max_features=5, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=500,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [49]:
gb_accuracy = gb.score(X_test, y_test)*100      
gb_accuracy 

73.1

In [50]:
df_test = pd.read_csv('test.csv')

In [51]:
df_test.columns = df_test.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [23]:
df_test.drop(columns = ['diplomatic_meetings_set','hostile_nations','aircraft_carriers_responding'],axis=1,inplace=True)

In [52]:
y_pred = gb.predict(df_test)

In [53]:
submission = pd.DataFrame({ 'ID': df_test.id.values, 'DEFCON_Level': y_pred })
submission.to_csv("subm2.csv", index=False)